In [98]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import statsmodels.stats.api as sms
import scipy as sp
import scipy.stats
import csv

def plot_run(path_to_dir):
    df = pd.read_csv(os.path.join(path_to_dir, 'stats_run.csv'))
    # print(df)
    for column in df.columns:
        plt.figure(figsize=(10,6), dpi=80)
        #plt.scatter(df['episode'], df[column])
        plt.plot(df['episode'], df[column],
                 label=column, color='blue', linewidth=2.0)
        plt.ylabel(column, fontsize=20, fontweight='bold')
        plt.xlabel('episodes', fontsize=20, fontweight='bold')
        plt.legend(fontsize=14)
        plt.savefig(os.path.join(path_to_dir, 'plot_' + str(column) + '.png'))
        plt.show()

def plot_runs(path_to_dir):
    run_dirs = glob.glob(os.path.join(path_to_dir) + '/*/')
    dfs = []
    for run_dir in run_dirs:
        dfs.append(pd.read_csv(os.path.join(run_dir, 'stats_run.csv')))
    for column in dfs[0].columns:
        plt.figure(figsize=(10, 6), dpi=80)
        run_count = 1
        for df in dfs:    
            plt.plot(df['episode'], df[column],
                     label=column+'_'+str(run_count), linewidth=2.0)
            run_count += 1
        plt.ylabel(column, fontsize=20, fontweight='bold')
        plt.xlabel('episodes', fontsize=20, fontweight='bold')
        plt.legend(fontsize=14)
        plt.savefig(os.path.join(path_to_dir, 'plot_' + str(column) + '.png'))
        plt.show()
            
def summarize_runs(path_to_dir):
    run_dirs = glob.glob(os.path.join(path_to_dir) + '/*/')
    run_files = [os.path.join(run_dir, 'stats_run.csv') for run_dir in run_dirs]
    df = pd.concat((pd.read_csv(run_file) for run_file in run_files))
    steps = df.groupby(['episode'])['steps_mean']
    steps = list(steps)
    reward = df.groupby(['episode'])['reward_mean']
    reward = list(reward)
    summary = []
    for episode in range(0, len(reward)):
        step_mean, step_lower, step_upper = mean_confidence_interval(steps[episode][1])
        reward_mean, reward_lower, reward_upper = mean_confidence_interval(reward[episode][1])
        summary.append([episode, step_mean, step_lower, step_upper, reward_mean, reward_lower, reward_upper])
    
    header = ['episode', 'steps_mean', 'steps_lower', 'steps_upper', 'reward_mean', 'reward_lower', 'reward_upper']
    try:
        with open(os.path.join(path_to_dir, 'stats_task1.csv'), 'w') as csvfile:
            writer = csv.writer(csvfile, dialect='excel', quoting=csv.QUOTE_NONNUMERIC)
            writer.writerow(header)
            for data in summary:
                writer.writerow(data)
    except IOError as e:
        print("Can't write stats file - %s " % str(e))
        #_logger.critical("Can't write stats file - %s " % str(e))
        # sys.exit()
    
def mean_confidence_interval(my_list, confidence=0.95):
    my_array = 1.0 * np.array(my_list)
    array_mean, array_se = np.mean(my_array), scipy.stats.sem(my_array)
    margin = array_se * sp.stats.t._ppf((1 + confidence) / 2.,
                                        len(my_array) - 1)
    return array_mean, array_mean - margin, array_mean + margin

def plot_task(path_to_dir):
    df = pd.read_csv(os.path.join(path_to_dir, 'stats_task.csv'))
    factors = ['steps', 'reward']
    colors = ['blue', 'green']
    for factor, color in zip(factors, colors):
        plt.figure(figsize=(10, 6), dpi=80)
        plt.plot(df['episode'], df[factor + '_mean'],
                 label=factor+'_mean', color=color, linewidth=2.0)
        plt.plot(df['episode'], df[factor + '_lower'],
                 label=factor+'_lower', color=color, linewidth=1.0)
        plt.plot(df['episode'], df[factor + '_upper'],
                 label=factor+'_upper', color=color, linewidth=1.0)
        plt.fill_between(df['episode'], df[factor + '_mean'],
                         df[factor + '_lower'],
                         facecolor=color, alpha=0.2)
        plt.fill_between(df['episode'], df[factor + '_mean'],
                         df[factor + '_upper'],
                         facecolor=color, alpha=0.2)
        plt.ylabel(factor, fontsize=20, fontweight='bold')
        plt.xlabel('episodes', fontsize=20, fontweight='bold')
        plt.legend(fontsize=14)
        plt.savefig(os.path.join(path_to_dir, 'plot_' + str(factor) + '.png'))
        plt.show()

In [99]:
path_to_dir = '/Users/rubenglatt/playground/CBPI/logs/2017-02-19_23-09_q_policyreuse2006/task_omega/'

#file_name = 'stats_run.csv'
#plot_runs(path_to_dir)
summarize_runs(path_to_dir)
plot_task(path_to_dir)

KeyError: 'Column not found: steps_mean'